In [ ]:

# Salience-Driven TD Learning Model with Gradient Descent Optimization

import numpy as np
import pandas as pd
from scipy.optimize import minimize
from concurrent.futures import ProcessPoolExecutor

# Define the salience-driven TD model
def td_model_salience(trial_data, s_t_vector, params, trial_outcome):
    base_alpha, gamma, a, b, beta, lambda_tone, lambda_shock, lambda_omission = params
    num_time_steps = len(trial_data)

    # Initialize arrays
    da_signal_trial = np.zeros(num_time_steps)
    value_function_trial = np.zeros(num_time_steps)
    td_error_trial = np.zeros(num_time_steps)

    # Initialize the first DA signal with the baseline
    da_signal_trial[0] = b

    # Compute TD error and DA signal for each time step
    for t in range(num_time_steps - 1):
        # Compute value function: V(t) = a * t + b
        value_function_trial[t] = a * t + b

        # External stimulus: S(t) from s_t_vector
        s_t = s_t_vector[t]

        # Salience-weighted learning rate
        salience_weight = (
            lambda_tone +
            lambda_shock * (trial_outcome == 1) +
            lambda_omission * (trial_outcome == 0)
        )

        # Compute TD error: δ(t) = α_salience(t) * (r_t + γ * V(t+1) - V(t))
        v_next = a * (t + 1) + b  # V(t+1)
        td_error_trial[t] = salience_weight * (s_t + gamma * v_next - value_function_trial[t])

        # Dynamic learning rate: α_t = base_alpha * (1 + beta * |δ(t)|)
        dynamic_alpha = base_alpha * (1 + beta * abs(td_error_trial[t]))

        # Update DA signal: Signal(t+1) = Signal(t) + α_t * δ(t)
        da_signal_trial[t + 1] = da_signal_trial[t] + dynamic_alpha * td_error_trial[t]

    return da_signal_trial, td_error_trial, value_function_trial

# Parallelize optimization for each trial
def optimize_td_model(trial_index):
    # Extract observed signal for the trial (replace with actual data loading)
    observed_signal = fiber_data.values[trial_index, :]

    # Create external stimulus vector (shock/avoid binary)
    s_t_vector = np.zeros(num_time_steps)
    if corrected_shock_outcomes[trial_index] == 1:
        s_t_vector[shock_start:shock_end] = 1  # Shock trials

    # Objective function for optimization
    def td_trial_error_salience(params):
        modeled_signal, _, _ = td_model_salience(observed_signal, s_t_vector, params, corrected_shock_outcomes[trial_index])
        return np.sum((observed_signal - modeled_signal) ** 2)  # Sum of squared errors

    # Initial guesses and bounds for the parameters
    initial_params = [0.1, 0.9, 0.05, 0.5, 0.1, 0.2, 0.8, -0.8]  # base_alpha, gamma, a, b, beta, λ_tone, λ_shock, λ_omission
    bounds = [(0.01, 1), (0.5, 1), (0, 0.1), (0, 1), (0, 1), (0, 1), (0, 2), (-2, 0)]  # Bounds for all parameters

    # Optimize parameters using gradient descent
    result = minimize(td_trial_error_salience, initial_params, bounds=bounds, method='L-BFGS-B')
    optimized_params = result.x

    # Compute modeled signal using optimized parameters
    modeled_signal, _, _ = td_model_salience(observed_signal, s_t_vector, optimized_params, corrected_shock_outcomes[trial_index])

    # Return trial results
    return trial_index, modeled_signal, optimized_params

# Use ProcessPoolExecutor for parallel optimization
with ProcessPoolExecutor() as executor:
    results = list(executor.map(optimize_td_model, range(num_trials)))

# Extract results
modeled_signals_salience_parallel = [res[1] for res in results]
optimized_parameters_salience_parallel = [res[2] for res in results]

# Create DataFrame of optimized parameters
optimized_parameters_salience_df_parallel = pd.DataFrame(
    optimized_parameters_salience_parallel,
    columns=["Base Alpha", "Gamma", "Uncertainty (a)", "Baseline (b)", "Beta (Dynamic α)", "λ_tone", "λ_shock", "λ_omission"]
)
optimized_parameters_salience_df_parallel["Trial"] = np.arange(1, num_trials + 1)

# Display results
optimized_parameters_salience_df_parallel.head()
    